In [46]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

In [47]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.5)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

In [48]:
examples = [
    {
        "movie": "top gun",
        "emoji": """🛩️👨‍✈️🔥"""
    },
    {
        "movie": "godfather",
        "emoji": """👨‍👨‍👦🔫🍝"""
    }
]

In [49]:
example_prompt = PromptTemplate(
    input_variables=["movie", "emoji"],
    template="Q: Reply with three emojis that represent the movie {movie}\nA: {emoji}"
)

In [50]:
suffix = (
    "Conversation so far (may be a summary):\n{history}\n\n"
    "You are a helpful assistant. If the user input is a question, answer in plain text using the conversation history when relevant.\n"
    "If the user input is a movie title or looks like a movie query, reply with exactly three emojis that represent it.\n"
    "Never include explanations when replying with emojis.\n\n"
    "User input: {movie}\n"
    "Answer:"
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix=suffix,
    input_variables=["movie", "history"]
)

In [51]:
chain = (
    RunnablePassthrough.assign(history=load_memory)
    | few_shot_prompt
    | llm
)

In [52]:
def invoke_chain(movie):
    result = chain.invoke({"movie": movie})
    memory.save_context({"input": movie}, {"output": result.content})
    print(result.content)

In [53]:
invoke_chain("toy story")

🧸🚀🤠


In [54]:
invoke_chain("inception")

🧠🕰️🚪


In [55]:
invoke_chain("what was the first movie I asked?")

toy story


In [56]:
invoke_chain("Which of these (🧸🤠🚀, 🕰️🧠🚪, 🚀🌌👨‍🚀) have you never mentioned before?")

🚀🌌👨‍🚀
